In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.display import display_markdown as mkdown # as print

def nl():
    print('\n')

for f in os.listdir('../input'):
    print(f.ljust(30) + str(round(os.path.getsize('../input/' + f) / 1000000, 2)) + 'MB')

In [ ]:
# copy from: https://www.kaggle.com/vykhand/grupo-bimbo-inventory-demand/exploring-products

In [ ]:
products  =  pd.read_csv("../input/producto_tabla.csv")
products  =  pd.read_csv("../input/producto_tabla.csv")
products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
products['weight'] = w[0].astype('float')*w[1].map({'Kg':1000, 'g':1})
products['pieces'] =  products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')
products.head()

In [ ]:
def lstm_model(time_steps, rnn_layers, dense_layers=None):
  """
  Creates a deep model based on:
      * stacked lstm cells
      * an optional dense layers
  :param time_steps: the number of time steps the model will be looking at.
  :param rnn_layers: list of int or dict
                       * list of int: the steps used to instantiate the `BasicLSTMCell` cell
                       * list of dict: [{steps: int, keep_prob: int}, ...]
  :param dense_layers: list of nodes for each layer
  :return: the model definition
  """

  def lstm_cells(layers):
    if isinstance(layers[0], dict):
      return [tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.BasicLSTMCell(layer['steps'],
                                                                        state_is_tuple=True),
                                                                        layer['keep_prob'])
      if layer.get('keep_prob') else tf.nn.rnn_cell.BasicLSTMCell(layer['steps'],
                                                                  state_is_tuple=True)
      for layer in layers]
    return [tf.nn.rnn_cell.BasicLSTMCell(steps, state_is_tuple=True) for steps in layers]

  def dnn_layers(input_layers, layers):
    if layers and isinstance(layers, dict):
      return learn.ops.dnn(input_layers,
                                layers['layers'],
                                activation=layers.get('activation'),
                                dropout=layers.get('dropout'))
    elif layers:
      return learn.ops.dnn(input_layers, layers)
    else:
      return input_layers

  def _lstm_model(X, y):
    stacked_lstm = tf.nn.rnn_cell.MultiRNNCell(lstm_cells(rnn_layers), state_is_tuple=True)
    x_ = learn.ops.split_squeeze(1, time_steps, X)
    output, layers = tf.nn.rnn(stacked_lstm, x_, dtype=dtypes.float32)
    output = dnn_layers(output[-1], dense_layers)
    return learn.models.linear_regression(output, y)

  return _lstm_model

In [ ]:
def rnn_data(data, time_steps, labels=False):
  """
  creates new data frame based on previous observation
    * example:
      l = [1, 2, 3, 4, 5]
      time_steps = 2
      -> labels == False [[1, 2], [2, 3], [3, 4]]
      -> labels == True [2, 3, 4, 5]
  """
  rnn_df = []
  for i in range(len(data) - time_steps):
    if labels:
      try:
        rnn_df.append(data.iloc[i + time_steps].as_matrix())
      except AttributeError:
        rnn_df.append(data.iloc[i + time_steps])
    else:
      data_ = data.iloc[i: i + time_steps].as_matrix()
      rnn_df.append(data_ if len(data_.shape) > 1 else [[i] for i in data_])
  return np.array(rnn_df)


def split_data(data, val_size=0.1, test_size=0.1):
  """
  splits data to training, validation and testing parts
  """
  ntest = int(round(len(data) * (1 - test_size)))
  nval = int(round(len(data.iloc[:ntest]) * (1 - val_size)))

  df_train, df_val, df_test = data.iloc[:nval], data.iloc[nval:ntest], data.iloc[ntest:]

  return df_train, df_val, df_test


def prepare_data(data, time_steps, labels=False, val_size=0.1, test_size=0.1):
  """
  Given the number of `time_steps` and some data,
  prepares training, validation and test data for an lstm cell.
  """
  df_train, df_val, df_test = split_data(data, val_size, test_size)
  return (rnn_data(df_train, time_steps, labels=labels),
           rnn_data(df_val, time_steps, labels=labels),
           rnn_data(df_test, time_steps, labels=labels))


def generate_data(fct, x, time_steps, seperate=False):
  """generates data with based on a function fct"""
  data = fct(x)
  if not isinstance(data, pd.DataFrame):
    data = pd.DataFrame(data)
    train_x, val_x, test_x = prepare_data(data['a'] if seperate else data, time_steps)
    train_y, val_y, test_y = prepare_data(data['b'] if seperate else data, time_steps, labels=True)
    return dict(train=train_x, val=val_x, test=test_x), dict(train=train_y, val=val_y, test=test_y)